# Graphing Inputs/Outputs of Pre and Post-Synaptic Neurons

## Import all relevent libraries

In [ ]:
!pip install seaborn --upgrade
!pip install standard-transform
import seaborn as sns
import random
from caveclient import CAVEclient
import nglui
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from phase3 import nda
from standard_transform import minnie_transform_vx
tform_vx = minnie_transform_vx()

## Using CAVEclient to query data

In [ ]:
client = CAVEclient()

In [ ]:
client.auth.save_token(token="a80f7690d93ead1b46e980628df5beae",overwrite = True)

In [ ]:
# this is the datastack name of the public release
# passing it will auto-configure many of the services
client = CAVEclient('minnie65_public_v343')

In [ ]:
#visualize available tables
client.materialize.get_tables()

## Filter Data to Obtain Neurons We Want to Observe

In [ ]:
df=client.materialize.query_table('functional_coreg') #dataframe of the neurons that have been proofread
df_functional = df.set_index('pt_root_id')

In [ ]:
df_functional.head() #visualize first couple neurons

In [ ]:
root_ids = df_functional.index.tolist() #extract column of IDs to loop over

In [ ]:
#we specify a subsection of neurons here so we can build our synapse data without causing the server to time out
subset_ids = root_ids[]

In [ ]:

x_values = []
y_values = []
for id in subset_ids:
    input_df = client.materialize.synapse_query(post_ids=id)
    
    input_df.rename(columns = {'pre_pt_root_id':'pt_root_id'}, inplace = True) 
   
    df_functional_matched = pd.merge(input_df,df_functional,on='pt_root_id',how='inner')
    if df_functional_matched.empty:
        continue
        
    #This is the correct depth value that is retrieved by a special function created by the MICrONS team    
    input_vertices_list = tform_vx.apply(df_functional_matched['pt_position'])
    graph_x_stack = np.vstack(input_vertices_list)
    graph_x_values = graph_x_stack[:,1]

    output_vertices = tform_vx.apply(df_functional.at[id,'pt_position'])
    graph_y_value = output_vertices[1]
    if type(graph_y_value) != np.float64:
        continue
    
    num_points = len(graph_x_values)
    x_values.extend(graph_x_values)
    y_values.extend([graph_y_value]*num_points)
    

In [ ]:
df_export = pd.DataFrame(list(zip(x_values, y_values)),
               columns =['X Values', 'Y Values'])

In [ ]:
df_export.to_csv('graph_values.csv')

In [ ]:
sns.jointplot(x=x_values, y=y_values, kind="hex", color="#4CB391")